In [1]:
# Imports
from airplane_reservoir import Model


import datetime
import os
import time

# Scientific computing imports
import numpy
import matplotlib.pyplot as plt
import networkx as nx
import pandas
import seaborn; seaborn.set()

# Import widget methods
from IPython.html.widgets import *

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
# Create our test model
# Initialize Space
g=nx.read_weighted_edgelist("2010citytocity_monthly.csv",nodetype=str,delimiter=',')
g=nx.convert_node_labels_to_integers(g,label_attribute='old_label')
g=g.to_directed()

m = Model(g)
for t in xrange(10):
    m.step()

In [3]:
print m.history_num_interactions
print m.time_series_data

[2, 4, 6, 3, 4, 3, 6, 1, 4, 1]
[(1000, 1, 0), (786, 3, 2), (614, 5, 4), (487, 9, 6), (366, 10, 3), (272, 10, 4), (213, 8, 3), (180, 11, 6), (120, 10, 1), (104, 13, 4), (68, 12, 1)]


In [4]:
os.chdir("C:\Users\colinsh\Box Sync\Winter 2015\IOE 513\IOE 513 Data")
os.listdir("./")

['2010citytocity_monthly.csv', 'data_vis', 'output', 'output_reservoir']

In [5]:
# Now, we'll define our methods to store a model's output

def store_model_parameters(model, run_output_path):
    """
    Store model parameters from a model to the run output path.
    """
    # Create parameters dictionary
    model_parameters = {"initial_infected": model.initial_infected,
                        "reproduction_number": model.reproduction_number,
                        "probability_of_flying_day": model.prob_of_flying_day,
                        "probability_of_flying": model.prob_of_flying,
                        "effective_contacts": model.effective_contacts,
                        "transmission_rate": model.transmission_rate,
                        "passengers_per_flight": model.passengers_per_flight,
                        "reservoir": model.reservoir,
                        "screening_effectiveness": model.screening_effectiveness,
                        "throttle": model.throttle,
                       }
    # Convert to dataframe and save
    model_parameters_df = pandas.DataFrame(model_parameters.items(),
                             columns=["parameter", "value"])
    model_parameters_df.to_csv(os.path.join(run_output_path, "parameters.csv"),index=False)
    

def store_model_csv(model, run_output_path):
    """
    Store CSV data from a model to the run output path.
    """
    # Create interaction dataframe
    try:
        interaction_df = pandas.DataFrame(model.history_infected_interactions,
                     columns=["time", "departure_city", "arrival_city", "departing_infected", "arriving_infected"])
    except ValueError:
        # Sometimes, we have no interactions in "sparse" parameter configurations.
        interaction_df = pandas.DataFrame(columns=["time", "departure_city", "arrival_city", "departing_infected", "arriving_infected"])
    
    # Create time series data frame
    tsdata_df = pandas.DataFrame(model.time_series_data,
                            columns=["num_infected","cities_infected","num_interactions"])
    #tsdata_df["num_interactions"] = model.history_num_interactions
    #tsdata_df["num_interactions_condoms"] = model.history_num_interactions_condoms
    
    # Save the dataframes
    interaction_df.to_csv(os.path.join(run_output_path, "interactions.csv"),index=False)
    tsdata_df.to_csv(os.path.join(run_output_path, "timeseries.csv"))

    
def store_model_figures(model, run_output_path):
    """
    Store figures data from a model to the run output path.
    """
    # Plot time series of infections and interactions.
    f = plt.figure(figsize=(10, 8))
    
    # Create our top panel
    plt.subplot(211)
    plt.plot(model.history_infected)
    plt.legend(('Number of infections',), loc="best")
    
    
    
    # Create our bottom panel and add the legend
    plt.subplot(212)
    plt.plot(model.history_num_interactions)
    plt.plot(model.history_cities_infected)
    plt.legend(("Number of interactions","Number of cities infected"), loc="best")
    plt.tight_layout()
    
    # Save
    plt.savefig(os.path.join(run_output_path, "infections_interactions.png"))
    
    # Next, plot the initial and final space timesteps.
    
    # Get colormap
    cmap = seaborn.cubehelix_palette(light=1, as_cmap=True)

    # Plot initial step.
    #f = plt.figure(figsize=(10, 10))
    #plt.title("Infected space at t={0}".format(0))
    #plt.pcolor(model.get_space_infected(0), vmin=-1, vmax=1, cmap=cmap)
    #ax = f.gca()
    #ax.set_aspect(1./ax.get_data_ratio())   
    #plt.tight_layout()
    #plt.colorbar()
    
    # Save
    #plt.savefig(os.path.join(run_output_path, "space_initial.png"))
    
    # Plot final step
    #plt.title("Infected space at t={0}".format(model.t-1))
    #plt.pcolor(model.get_space_infected(model.t-1), vmin=-1, vmax=1, cmap=cmap)
    #ax = f.gca()
    #ax.set_aspect(1./ax.get_data_ratio())   
    #plt.tight_layout()
    #plt.colorbar()
    
    # Save
    #plt.savefig(os.path.join(run_output_path, "space_final.png"))    
    

def store_model(model, output_path="output"):
    """
    Store a model to the model output path.
    """
    # First, we need to make sure the directory exists.
    try:
        os.makedirs(output_path)
    except:
        pass
    
    """
    Next, we need to create a unique timestamp for the model.
    We'll do that using a timestamp of the form: YYYYMMDD-Run#
    
    We then need to create that directory too.
    """
    
    timestamp_suffix = time.strftime("%Y%m%d")
    
    run_id = 0
    run_output_path = os.path.join(output_path,
                                 "run-{0}-{1}".format(timestamp_suffix,
                                                     run_id))
    # Get a unique run #
    while os.path.exists(run_output_path):
        run_id += 1
        run_output_path = os.path.join(output_path,
                                 "run-{0}-{1}".format(timestamp_suffix,
                                                     run_id))        

    try:
        os.makedirs(run_output_path)
    except:
        pass
    
    """
    Finally, we need to store data and figures to the path.
    """
    store_model_parameters(model, run_output_path)
    store_model_csv(model, run_output_path)
    #store_model_figures(model, run_output_path)

In [6]:
#Running Parameter Sweep
# Initialize Space
g=nx.read_weighted_edgelist("2010citytocity_monthly.csv",nodetype=str,delimiter=',')
g=nx.convert_node_labels_to_integers(g,label_attribute='old_label')
g=g.to_directed()

# Set sweep parameters
initial_infection_sweep_values=[500,1000,1500]
reproduction_number_sweep_values=[.6,.7,.8,.9,1]
res=[0,1]
screening=[0,.3,.6]
throttle=[0,.2,.4]

num_samples=20
num_steps=18

# Iterate over initial infection
for infection in initial_infection_sweep_values:
    # Iterate over reproduction number
    for r_0 in reproduction_number_sweep_values:
        # Iterate over reservoir
        for r in res:
            #Iterate over screening
            for screen in screening:
                #Iterate over throttle
                for throt in throttle:      
                    print("Running {0} samples for initial infection={1}, reproduction number={2}, reservoir={3}, screening effectivness={4}, throttling={5}"
                          .format(num_samples,infection,r_0,r,screen,throt))
                    for n in xrange(num_samples):
                        # Initialize Space
                        g=nx.read_weighted_edgelist("2010citytocity_monthly.csv",nodetype=str,delimiter=',')
                        g=nx.convert_node_labels_to_integers(g,label_attribute='old_label')
                        g=g.to_directed()

                        # Output info
                        m=Model(g,reproduction_number=r_0,initial_infected=infection,reservoir=r,throttle=throt,screening_effectiveness=screen)

                        # Run the modle for num_steps
                        for t in xrange(num_steps):
                            m.step()

                        #Output the model
                        store_model(m,output_path="output_reservoir_screen")

        
        
        

Running 20 samples for initial infection=500, reproduction number=0.6, reservoir=0, screening effectivness=0, throttling=0
Running 20 samples for initial infection=500, reproduction number=0.6, reservoir=0, screening effectivness=0, throttling=0.2
Running 20 samples for initial infection=500, reproduction number=0.6, reservoir=0, screening effectivness=0, throttling=0.4
Running 20 samples for initial infection=500, reproduction number=0.6, reservoir=0, screening effectivness=0.3, throttling=0
Running 20 samples for initial infection=500, reproduction number=0.6, reservoir=0, screening effectivness=0.3, throttling=0.2
Running 20 samples for initial infection=500, reproduction number=0.6, reservoir=0, screening effectivness=0.3, throttling=0.4
Running 20 samples for initial infection=500, reproduction number=0.6, reservoir=0, screening effectivness=0.6, throttling=0
Running 20 samples for initial infection=500, reproduction number=0.6, reservoir=0, screening effectivness=0.6, throttling=

In [6]:
1==[0,1]


False